In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import random

# Định nghĩa đường dẫn cho ChromeDriver
CHROME_DRIVER_PATH = "chromedriver.exe" 

# URL gốc của trang báo cáo lương
BASE_URL = "https://vietnamsalary.careerviet.vn"
REPORT_URL = "https://vietnamsalary.careerviet.vn/industry-report-1-vi"

In [2]:
# Hàm trích xuất lương tổng quan từ khối "statistic-area" (KHÔNG CẦN THAY ĐỔI)
def extract_overview_salary(detail_soup):
    """Trích xuất lương Trung bình, Thấp nhất và Cao nhất từ khối tổng quan."""
    
    overview_data = {
        'Overview_Avg': 'N/A',
        'Overview_Min': 'N/A',
        'Overview_Max': 'N/A',
    }
    
    # Tìm khối chứa tất cả các chỉ số thống kê
    stats_block = detail_soup.find('div', class_='statistic-area')
    
    if stats_block:
        # 1. Lương Trung bình (Item 1)
        avg_block = stats_block.find('div', class_='item-1')
        if avg_block:
            avg_tag = avg_block.find('strong')
            if avg_tag:
                overview_data['Overview_Avg'] = avg_tag.text.strip()
        
        # 2. Lương Thấp nhất (Nằm trong Item 2, row-data-2)
        min_block = stats_block.find('div', class_='item-2')
        if min_block:
            low_data_row = min_block.find('div', class_='row-data-2')
            if low_data_row:
                min_tag = low_data_row.find('strong')
                if min_tag:
                    overview_data['Overview_Min'] = min_tag.text.strip().replace('/ tháng', '')
        
        # 3. Lương Cao nhất (Nằm trong Item 3, row-data-2)
        max_block = stats_block.find('div', class_='item-3')
        if max_block:
            high_data_row = max_block.find('div', class_='row-data-2')
            if high_data_row:
                max_tag = high_data_row.find('strong')
                if max_tag:
                    overview_data['Overview_Max'] = max_tag.text.strip().replace('/ tháng', '')
                    
    return overview_data

In [ ]:
# Hàm để trích xuất dữ liệu lương chi tiết từ MỘT trang báo cáo
def scrape_detail_salary(driver, job_url, job_title):
    """
    Truy cập trang chi tiết báo cáo và CHỈ CÀO các chỉ số tổng quan (Min/Max/Avg).
    """
    print(f"  > Đang cào tổng quan: {job_title}")
    driver.get(job_url)
    time.sleep(random.uniform(3, 5)) 

    detail_soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    overview_stats = extract_overview_salary(detail_soup)
    
    if overview_stats['Overview_Avg'] != 'N/A':
        # Trả về một bản ghi duy nhất cho chức danh này
        return [{
            'Job_Title_Raw': job_title,
            'Salary_Min_Overview': overview_stats['Overview_Min'],
            'Salary_Max_Overview': overview_stats['Overview_Max'],
            'Salary_Avg_Overview': overview_stats['Overview_Avg'],
            'Source_Link': job_url
        }]
    
    return []

In [ ]:
# --- Quy trình Cào Chính ---
def scrape_vietnamsalary():
    try:
        service = Service(CHROME_DRIVER_PATH)
        driver = webdriver.Chrome(service=service)
    except Exception as e:
        print(f"LỖI NGHIÊM TRỌNG: Không thể khởi tạo Chrome WebDriver. Lỗi: {e}")
        return

    all_salary_data = []

    # BƯỚC 1: CÀO DANH SÁCH CHỨC DANH VÀ LINK TỔNG QUAN
    print("--- BƯỚC 1: CÀO DANH SÁCH CHỨC DANH VÀ LINK TỔNG QUAN ---")
    driver.get(REPORT_URL)
    time.sleep(random.uniform(5, 8)) # Đợi trang tải đủ lâu

    main_soup = BeautifulSoup(driver.page_source, 'html.parser')
    job_links = []

    try:
        # Dùng BeautifulSoup để tìm tất cả các thẻ <a> có thuộc tính href chứa '/detail/'
        all_link_tags = main_soup.find_all('a', href=lambda href: href and '/detail/' in href)
        
        for tag in all_link_tags:
            link = tag.get('href')
            title = tag.text.strip().lower() 
            
            if link and title:
                full_url = f"{BASE_URL}{link}" if not link.startswith('http') else link
                job_links.append({'title': title, 'url': full_url})
                
        job_links = [dict(t) for t in {tuple(d.items()) for d in job_links}] # Loại bỏ link trùng lặp


        if not job_links:
             print("KHÔNG THỂ TÌM THẤY CÁC LINK CHI TIẾT BÁO CÁO LƯƠNG. Vui lòng kiểm tra lại cấu trúc trang web.")
             driver.quit()
             return

    except Exception as e:
        print(f"LỖI CÀO DANH SÁCH LINK: {e}")
        driver.quit()
        return

    # BƯỚC 2: CÀO CHI TIẾT TỪNG TRANG
    print(f"\n--- BƯỚC 2: CÀO CHI TIẾT {len(job_links)} CHỨC DANH ---")
    for job in job_links:
        detail_data = scrape_detail_salary(driver, job['url'], job['title'])
        all_salary_data.extend(detail_data)
        time.sleep(random.uniform(1, 3))

    driver.quit()

    # BƯỚC 3: LƯU DỮ LIỆU
    if all_salary_data:
        df = pd.DataFrame(all_salary_data)
        file_name = 'vietnamsalary_report_overview.csv'
        df.to_csv(file_name, index=False, encoding='utf-8-sig')
        print(f"\nHoàn tất cào dữ liệu lương tổng quan và lưu vào file '{file_name}'")
        print(f"Tổng số bản ghi lương đã cào: {len(df)}")
    else:
        print("\nKhông có dữ liệu lương tổng quan nào được cào thành công.")

In [5]:
if __name__ == "__main__":
    scrape_vietnamsalary()

--- BƯỚC 1: CÀO DANH SÁCH CHỨC DANH VÀ LINK TỔNG QUAN ---

--- BƯỚC 2: CÀO CHI TIẾT 72 CHỨC DANH ---
  > Đang cào tổng quan: ui/ux designer
  > Đang cào tổng quan: lập trình viên frontend
  > Đang cào tổng quan: android developer
  > Đang cào tổng quan: nhân viên quản trị hệ thống
  > Đang cào tổng quan: nhân viên seo
  > Đang cào tổng quan: trưởng phòng it
  > Đang cào tổng quan: it business analyst
  > Đang cào tổng quan: lương nhân viên kinh doanh
  > Đang cào tổng quan: product owner
  > Đang cào tổng quan: lương kế toán trưởng
  > Đang cào tổng quan: it developer
  > Đang cào tổng quan: lương sales executive
  > Đang cào tổng quan: senior .net developer
  > Đang cào tổng quan: nhân viên hỗ trợ kỹ thuật
  > Đang cào tổng quan: it officer
  > Đang cào tổng quan: it staff
  > Đang cào tổng quan: data analyst
  > Đang cào tổng quan: senior backend developer
  > Đang cào tổng quan: lập trình viên ios
  > Đang cào tổng quan: nhân viên phân tích nghiệp vụ
  > Đang cào tổng quan: lương ac